In [59]:
from datetime import datetime
import pandas as pd
import warnings
import math
import sys
sys.path.append('../src')
from functions import stock_selection_weight_allocation, adjust_portfolio
warnings.filterwarnings("ignore")

In [60]:
def custom_round(number):
    # Separate the number into the integer and decimal parts
    integer_part = int(number)
    decimal_part = number - integer_part
    
    # Check if the decimal part is above .75; if so, round up
    if decimal_part > 0.8:
        return integer_part + 1
    else:
        return integer_part
    
def calculate_shares_to_buy_with_prices(portfolio_weights, prices_df, buying_date, total_investment, strictly_lower):

    # Step 1: Calculate initial amount to be invested in each stock
    initial_investment = {stock: weight * total_investment for stock, weight in portfolio_weights.items()}
    
    # Step 2: Determine share price for each stock on the day before buying
    day_before_buying_date = prices_df.index[prices_df.index.get_loc(buying_date) - 1]
    share_prices = prices_df.loc[day_before_buying_date]
    
    # Step 3: Calculate initial number of shares to buy for each stock
    if strictly_lower:
        initial_shares = {stock: math.floor(initial_investment[stock]/share_prices[stock]) for stock in portfolio_weights.keys()}
    else:
        initial_shares = {stock: custom_round(initial_investment[stock]/share_prices[stock]) for stock in portfolio_weights.keys()}

    # Step 4: Adjust for stocks that are too expensive
    affordable_stocks = {stock: shares for stock, shares in initial_shares.items() if shares >= 1}
    total_weight_of_affordable_stocks = sum([portfolio_weights[stock] for stock in affordable_stocks.keys()])
    adjusted_weights = {stock: portfolio_weights[stock] / total_weight_of_affordable_stocks for stock in affordable_stocks.keys()}
    adjusted_investment = {stock: adjusted_weights[stock] * total_investment for stock in affordable_stocks.keys()}
    
    # Step 5: Recalculate the number of shares to buy for each of the remaining stocks
    if strictly_lower:
        final_shares = {stock: math.floor(adjusted_investment[stock]/share_prices[stock]) for stock in affordable_stocks.keys()}
    else:
        final_shares = {stock: custom_round(adjusted_investment[stock]/share_prices[stock]) for stock in affordable_stocks.keys()}

    # Collect the buying prices for the affordable stocks
    price_dict = {stock: share_prices[stock] for stock in affordable_stocks.keys()}
    
    # Calculate actual investment based on final shares and their buying prices
    actual_investment = sum([final_shares[stock] * price_dict[stock] for stock in final_shares.keys()])
    
    return final_shares, price_dict, actual_investment

In [61]:
buy_date_str = '2024-02-24'
buy_date = pd.to_datetime(buy_date_str)

stock_selection_strategy = 13
weight_allocation_strategy = 6

# backtest version 5
last_x_years = 0.5
last_x_years_opt = 0.5

In [62]:
all_stocks_df = pd.read_csv('../data/all_stock_data.csv', index_col=0)
all_stocks_df.index = pd.to_datetime(all_stocks_df.index)
all_stocks_df

,360ONE.NS,3MINDIA.NS,ABB.NS,ACC.NS,AGI.NS,AIAENG.NS,APLAPOLLO.NS,AUBANK.NS,AARTIDRUGS.NS,AARTIIND.NS,...,ZEEL.NS,ZENTEC.NS,ZENSARTECH.NS,ZOMATO.NS,ZYDUSLIFE.NS,ZYDUSWELL.NS,ECLERX.NS,EMUDHRA.NS,ACLGATI.NS,WELSPUNLIV.NS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,264.346344,1779.729614,650.601746,722.450684,9.885830,333.858276,12.891179,282.469299,10.365798,8.898396,...,105.713058,53.752098,24.538420,126.000000,77.028969,243.609695,120.262016,256.129639,57.730717,7.873486
2010-01-05,264.346344,1779.393066,648.944153,713.079895,10.383352,343.307556,12.891179,282.469299,10.268604,8.898396,...,106.864403,53.752098,24.083796,126.000000,79.013733,243.020615,126.622154,256.129639,57.596756,7.687500
2010-01-06,264.346344,1769.392090,649.156616,717.706116,10.318739,333.007385,12.891179,282.469299,10.419255,8.880511,...,109.229851,53.752098,24.190123,126.000000,79.698532,241.162704,126.735474,256.129639,57.105633,7.819755
2010-01-07,264.346344,1783.816528,664.966919,722.252991,10.835644,322.438599,12.891179,282.469299,10.258885,8.817906,...,109.292633,53.752098,24.237785,126.000000,79.750763,238.217316,126.735474,256.129639,57.418175,7.699896
2010-01-08,264.346344,1779.585327,676.612183,721.304138,11.171633,331.081726,12.891179,282.469299,10.560185,8.853681,...,107.659851,53.752098,23.944483,126.000000,80.215050,237.537567,133.463913,256.129639,57.775345,7.766028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-13,691.700012,31471.150391,4464.850098,2614.600098,782.200012,3969.250000,1333.599976,593.799988,528.250000,662.849976,...,188.649994,831.849976,525.000000,154.750000,857.799988,1569.650024,2395.199951,471.700012,114.400002,152.500000
2024-02-14,707.849976,30947.250000,4558.399902,2619.600098,784.950012,3931.550049,1388.550049,590.750000,534.900024,667.799988,...,200.199997,826.400024,520.000000,152.250000,867.799988,1592.050049,2363.850098,470.350006,117.650002,150.399994
2024-02-15,713.599976,30788.349609,4468.600098,2637.649902,778.250000,3846.100098,1418.599976,581.900024,535.950012,681.950012,...,187.899994,829.650024,532.000000,154.850006,878.950012,1577.800049,2318.050049,475.399994,116.800003,153.300003


In [63]:
govt_bond_df = pd.read_csv('../data/India 10-Year Bond Yield Historical Data.csv')
govt_bond_df = govt_bond_df[['Date','Price']]
govt_bond_df.index = govt_bond_df['Date']
govt_bond_df = govt_bond_df.drop('Date', axis=1)
govt_bond_df.index = pd.to_datetime(govt_bond_df.index)
govt_bond_df

,Price
Date,
2024-01-01,7.196
2023-12-29,7.176
2023-12-28,7.207
2023-12-27,7.205
2023-12-26,7.183
...,...
2020-01-07,6.550
2020-01-06,6.565
2020-01-03,6.510


In [64]:
filters = 4

if stock_selection_strategy in [1,2,5,6,9,10,13,14]:
    holding_period = '1q'
elif stock_selection_strategy in [3,4,7,8,11,12,15,16]:
    holding_period = '1m'

if stock_selection_strategy in [1,3,5,7,9,11,13,15]:
    returns_type = 'SR'
elif stock_selection_strategy in [2,4,6,8,10,12,14,16]:
    returns_type = 'LR'

if stock_selection_strategy in [1,2,3,4]:
    max_non_positive_returns_count = 15
elif stock_selection_strategy in [5,6,7,8]:
    max_non_positive_returns_count = 10
elif stock_selection_strategy in [9,10,11,12]:
    max_non_positive_returns_count = None
    filters = 3
elif stock_selection_strategy in [13,14,15,16]:
    max_non_positive_returns_count = None
    filters = 2

In [65]:
portfolio, sell_date, best_method = stock_selection_weight_allocation(buy_date_str, holding_period, returns_type, max_non_positive_returns_count, weight_allocation_strategy, all_stocks_df, govt_bond_df, filters, last_x_years, last_x_years_opt)        
portfolio = adjust_portfolio(portfolio)
portfolio

{'ANANDRATHI.NS': 0.125,
 'SWANENERGY.NS': 0.125,
 'GET&D.NS': 0.125,
 'SUZLON.NS': 0.125,
 'BSE.NS': 0.125,
 'SOBHA.NS': 0.125,
 'TATAINVEST.NS': 0.125,
 'MCX.NS': 0.125}

In [66]:
total_investment = 10000

In [67]:
adjusted_buy_date = all_stocks_df[all_stocks_df.index <= buy_date].index[-1]

In [68]:
final_shares, price_dict, actual_investment = calculate_shares_to_buy_with_prices(portfolio, all_stocks_df, adjusted_buy_date, total_investment, strictly_lower=True)

In [69]:
final_shares, price_dict, actual_investment

({'SWANENERGY.NS': 4, 'GET&D.NS': 4, 'SUZLON.NS': 71},
 {'SWANENERGY.NS': 729.1500244140625,
  'GET&D.NS': 782.7000122070312,
  'SUZLON.NS': 46.84999847412109},
 9373.750038146973)